In [19]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup    
import re
from tqdm import tqdm_notebook

In [4]:
df = pd.read_csv('all_auto_ru_09_09_2020.csv', sep = ',')

In [5]:
df.sample(5)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
76768,Седан,VOLVO,97948F,бензин,1999.0,1.8 AT (122 л.с.),4.0,2003,SEDAN AUTOMATIC 1.8,AUTOMATIC,...,LEFT,NaN,3.0,DUPLICATE,True,NaN,125000.0,2020-07-20T07:48:33Z,NaN,S40
66214,Седан,TOYOTA,CACECB,бензин,2000.0,1.4 MT (97 л.с.),4.0,2003,SEDAN MECHANICAL 1.4,MECHANICAL,...,LEFT,NaN,2.0,ORIGINAL,True,NaN,305000.0,2020-09-08T05:49:42Z,NaN,COROLLA
10995,Внедорожник 5 дв.,BMW,0000CC,дизель,2019.0,40d 3.0d AT (340 л.с.) 4WD,5.0,2020,ALLROAD_5_DOORS AUTOMATIC 40d,AUTOMATIC,...,LEFT,NaN,NaN,ORIGINAL,True,NaN,7143400.0,2020-09-02T04:19:11Z,NaN,X6
49494,Внедорожник 5 дв.,NISSAN,040001,бензин,2010.0,2.0 MT (141 л.с.),5.0,2011,ALLROAD_5_DOORS MECHANICAL 2.0,MECHANICAL,...,LEFT,NaN,2.0,ORIGINAL,True,NaN,700000.0,2020-06-18T14:39:05Z,NaN,QASHQAI
33494,Универсал 5 дв.,KIA,0000CC,бензин,2012.0,1.6 AT (129 л.с.),5.0,2013,WAGON_5_DOORS AUTOMATIC 1.6,AUTOMATIC,...,LEFT,NaN,3.0,ORIGINAL,True,NaN,650000.0,2020-02-13T10:46:29Z,NaN,CEED


# Библиотека Sweetviz позволяет сгенерировать отдельный HTML документ с необходимым EDA

In [6]:
import sweetviz as sv

my_report = sv.analyze(df)
my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"

                                             |                                             | [  0%]   00:00 ->…

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


# Profile создает весь необходимый EDA внутри ноутбука

In [7]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, title="Pandas Profiling Report")

In [8]:
profile

Summarize dataset:   0%|          | 0/39 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [20]:
START_URL='https://auto.ru/moskva/cars/used/'

pages_url_list=[]

In [21]:
for i in tqdm_notebook(range(1,100)):   
    response = r.get( START_URL+'?page=%s'%i)
    page = BeautifulSoup(response.text, 'html.parser')
    for link in page.findAll('a',{"class": "Link OfferThumb"}):
        try:
            if link['href'] not in pages_url_list: 
                pages_url_list.append(link['href'])
        except:
            pass

<ipython-input-21-9289ee5f8e82>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1,100)):


  0%|          | 0/99 [00:00<?, ?it/s]

In [22]:
len(pages_url_list)

3147

In [23]:
cars_list = []
for item in tqdm_notebook(pages_url_list):
    response = r.get(item)
    response.encoding = 'utf8'
    page = BeautifulSoup(response.text, 'html.parser')
    soup_name = page.find('div', class_='CardBreadcrumbs').find_all('a')
    
    cls_str = 'CardInfoRow_'
    RE_INT = re.compile(r'^[-+]?([1-9]\d*|0)$')
    
    cars_list.append({'Brand':soup_name[2].text, 
                    'Mark':soup_name[3].text, 
                    'bodyType':page.find('li', class_=cls_str+'bodytype').find('a').text, 
                    'car_url':item,
                    'color':page.find('li', class_=cls_str+'color').find('a').text,
                    'engineVolume':page.find('li', class_=cls_str+'engine').find('div').text.split('/')[0],
                    'enginePower':page.find('li', class_=cls_str+'engine').find('div').text.split('/')[1],
                    'fuelType':page.find('li', class_=cls_str+'engine').find('div').text.split('/')[2],
                    'modelDate':page.find('li', class_=cls_str+'year').find('a').text,
                    'mileAge':[''.join(re.findall(r'\d+', 
                                                  page.find('li', class_=cls_str+'kmAge').find_all('span')[1].text))][0],
                    'crashes':page.find('div', class_='CardBenefits__item-title').text})
                    
# <div class="CardBenefits__item-title">ДТП не&nbsp;найдены</div>
# <span class="Link VinReportFreeBlockItem__text" tabindex="0" role="button">Данные о ДТП не найдены</span>
    
    print(page.find('div', class_='Link VinReportFreeBlockItem__text').text)

<ipython-input-23-e7f63ee5977e>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for item in tqdm_notebook(pages_url_list):


  0%|          | 0/3147 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'text'

In [99]:
cars_list = []
for item in tqdm_notebook(pages_url_list):
# item = pages_url_list[1]
    response = r.get(item)
    response.encoding = 'utf8'
    page = BeautifulSoup(response.text, 'html.parser')
    if page.find('div', class_='CardSold') == None:
        try:
            json_data = json.loads(
                page.find('script', type="application/ld+json").string)
        except:
            print(item)
            pass
        cls_str = 'CardInfoRow_'
        span_str = 'CardInfoRow__cell'
        try:
            cars_list.append({
            'bodyType': json_data['bodyType'],
            'brand': json_data['brand'],
            'car_url': json_data['offers']['url'],
            'color': json_data['color'],
            'description': json_data['description'],
            'engineDisplacement': json_data['vehicleEngine']['engineDisplacement'],
            'enginePower': json_data['vehicleEngine']['enginePower'],
            'fuelType': json_data['fuelType'],
            'image': json_data['image'],
            'mileage': page.find(
                'li', class_=cls_str+'kmAge').find_all('span')[1].text,
            'modelDate': json_data['modelDate'],
            'model_name': json_data['name'],
            'name': json_data['vehicleEngine']['name'],
            'numberOfDoors': json_data['numberOfDoors'],
            'parsing_unixtime': int(time.time()),
            'priceCurrency': json_data['offers']['priceCurrency'],
            'productionDate': json_data['productionDate'],
            'sell_id': page.find(
                'div', title='Идентификатор объявления').text,
            'vehicleConfiguration': json_data['vehicleConfiguration'],
            'vehicleTransmission': json_data['vehicleTransmission'],
            'Владельцы': page.find(
                'li', class_=cls_str+'ownersCount').find_all('span')[1].text,
            'ПТС': page.find(
                'li', class_=cls_str+'pts').find_all('span')[1].text,
            'Привод': page.find(
                'li', class_=cls_str+'drive').find_all('span')[1].text,
            'Руль': page.find(
                'li', class_=cls_str+'wheel').find_all('span')[1].text,
            'Состояние': page.find(
                'li', class_=cls_str+'state').find_all('span')[1].text,
            'Таможня': page.find(
                'li', class_=cls_str+'customs').find_all('span')[1].text,
            'price': page.find(
                'span', class_='OfferPriceCaption__price').text
            })
        except:
            print(item)
            pass

   # else  = []

    #print('Ожидаю 0.1 секунды...')
#     time.sleep(0.1)
    print(cars_list)

<ipython-input-99-83fb5a8755e7>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for item in tqdm_notebook(pages_url_list):


  0%|          | 0/3305 [00:00<?, ?it/s]

https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1103632348-44140331/
https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1103632348-44140331/
[]
https://auto.ru/cars/used/sale/vaz/2112/1103701429-2d05a763/
https://auto.ru/cars/used/sale/vaz/2112/1103701429-2d05a763/
[]
[]
https://auto.ru/cars/used/sale/nissan/gt_r/1103644864-62565c8c/
https://auto.ru/cars/used/sale/nissan/gt_r/1103644864-62565c8c/
[]
https://auto.ru/cars/used/sale/citroen/ds5/1103748821-756c3a6e/
https://auto.ru/cars/used/sale/citroen/ds5/1103748821-756c3a6e/
[]
https://auto.ru/cars/used/sale/bentley/bentayga/1103732158-891d3ea3/
https://auto.ru/cars/used/sale/bentley/bentayga/1103732158-891d3ea3/
[]
https://auto.ru/cars/used/sale/nissan/gt_r/1103132013-f336cca8/
https://auto.ru/cars/used/sale/nissan/gt_r/1103132013-f336cca8/
[]
https://auto.ru/cars/used/sale/volvo/xc90/1103798840-db58b46c/
https://auto.ru/cars/used/sale/volvo/xc90/1103798840-db58b46c/
[]
https://auto.ru/cars/used/sale/vaz/2121/1103731925

KeyboardInterrupt: 

In [94]:
# переводим в удобный для нас вид
df = pd.DataFrame(cars_list)
# проверяем на дубликаты, можно по двум признакам car_url или sell_id
#df['sell_id'].duplicated().sum()
# сохраняем в csv
#df.to_csv('cars_df.csv', index=False)

In [24]:
cars_list

[]